## Prerequisites

To execute this tutorial you will need:

* duckdb
* boto3

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

In [3]:
import duckdb
import boto3
import pandas

## 1. Create a DuckDB Connection

Open a DuckDB database which is in memory by default

In [4]:
conn = duckdb.connect()

## 2. Install Iceberg ext. + Load it to this session

In [5]:
conn.execute("INSTALL 'iceberg'")
conn.execute("LOAD 'iceberg'")

## 3. Create Storage Credentials Secret

In [6]:
conn.execute("""
    CREATE OR REPLACE SECRET (
        TYPE s3,
        PROVIDER credential_chain
    )
""")

## 4. Attach the Glue Data Catalog 

Attach an external catalog ie. the Glue Data Catalog which contains our Iceberg table(s)

In [7]:
# Get current AWS account id
sts = boto3.client("sts")
account_id = sts.get_caller_identity()["Account"]
print(account_id)

851725370941


In [8]:
attach_sql = f"""
            ATTACH '{account_id}'
            AS gdc_catalog (
                TYPE iceberg,
                ENDPOINT 'glue.us-east-1.amazonaws.com/iceberg',
                AUTHORIZATION_TYPE 'sigv4'
            )
        """

conn.execute(attach_sql)

## 5. Read an Iceberg Table

List all tables

In [9]:
tables = conn.execute("SHOW ALL TABLES").fetchall()
print(tables)

[('gdc_catalog', 'iceberg', 'sampledataicebergtable', ['__'], ['UNKNOWN'], False)]


SELECT * FROM Iceberg Table

In [10]:
select = conn.execute("SELECT * FROM gdc_catalog.iceberg.sampledataicebergtable").fetchall()
print(select)

[('6', 'Adam', '2020-01-01', datetime.datetime(2020, 1, 1, 0, 0, tzinfo=<StaticTzInfo 'Etc/UTC'>)), ('2', 'Will', '2020-01-01', datetime.datetime(2020, 1, 1, 0, 0, tzinfo=<StaticTzInfo 'Etc/UTC'>)), ('1', 'Christopher', '2020-01-01', datetime.datetime(2020, 1, 2, 0, 0, tzinfo=<StaticTzInfo 'Etc/UTC'>)), ('3', 'Emmeline', '2020-01-01', datetime.datetime(2020, 1, 2, 0, 0, tzinfo=<StaticTzInfo 'Etc/UTC'>)), ('4', 'John', '2020-01-01', datetime.datetime(2020, 1, 1, 0, 0, tzinfo=<StaticTzInfo 'Etc/UTC'>))]


Send the results of the query to a Pandas dataframe

In [11]:
select = conn.execute("SELECT * FROM gdc_catalog.iceberg.sampledataicebergtable").df()
print(select.sort_values("id"))

  id         name create_date          last_update_time
2  1  Christopher  2020-01-01 2020-01-02 00:00:00+00:00
1  2         Will  2020-01-01 2020-01-01 00:00:00+00:00
3  3     Emmeline  2020-01-01 2020-01-02 00:00:00+00:00
4  4         John  2020-01-01 2020-01-01 00:00:00+00:00
0  6         Adam  2020-01-01 2020-01-01 00:00:00+00:00


Send the results to PyArrow

In [12]:
select = conn.execute("SELECT * FROM gdc_catalog.iceberg.sampledataicebergtable").arrow().read_all()
print(select)

pyarrow.Table
id: string
name: string
create_date: string
last_update_time: timestamp[us, tz=Etc/UTC]
----
id: [["6","2","1","3","4"]]
name: [["Adam","Will","Christopher","Emmeline","John"]]
create_date: [["2020-01-01","2020-01-01","2020-01-01","2020-01-01","2020-01-01"]]
last_update_time: [[2020-01-01 00:00:00.000000Z,2020-01-01 00:00:00.000000Z,2020-01-02 00:00:00.000000Z,2020-01-02 00:00:00.000000Z,2020-01-01 00:00:00.000000Z]]


## 6. Append to an Iceberg Table

In [13]:
table_name = "gdc_catalog.iceberg.sampledataicebergtable"

In [14]:
# Get table schema
schema = conn.execute(f"DESCRIBE {table_name}").fetchall()
print(schema)

[('id', 'VARCHAR', 'YES', None, None, None), ('name', 'VARCHAR', 'YES', None, None, None), ('create_date', 'VARCHAR', 'YES', None, None, None), ('last_update_time', 'TIMESTAMP WITH TIME ZONE', 'YES', None, None, None)]


In [15]:
# Insert new records matching the 4-column schema
insert_sql = f"""
    INSERT INTO {table_name} (id, name, create_date, last_update_time)
        VALUES 
            ('10', 'Sam', '2024-01-01', CURRENT_TIMESTAMP),
            ('11', 'Jacob', '2024-01-01', CURRENT_TIMESTAMP)
"""

conn.execute(insert_sql)

In [16]:
# Verify insert
results = conn.execute(f"SELECT * FROM {table_name}").df()
print(results.sort_values("id"))

   id         name create_date                 last_update_time
2   1  Christopher  2020-01-01        2020-01-02 00:00:00+00:00
5  10          Sam  2024-01-01 2025-12-17 17:52:02.547987+00:00
6  11        Jacob  2024-01-01 2025-12-17 17:52:02.547987+00:00
1   2         Will  2020-01-01        2020-01-01 00:00:00+00:00
3   3     Emmeline  2020-01-01        2020-01-02 00:00:00+00:00
4   4         John  2020-01-01        2020-01-01 00:00:00+00:00
0   6         Adam  2020-01-01        2020-01-01 00:00:00+00:00


## 7. Upsert existing records in Iceberg Table

In [17]:
update_sql = f"""
    UPDATE {table_name} 
    SET name = 'Sam-Update', create_date = '2024-01-01' , last_update_time = CURRENT_TIMESTAMP
    WHERE id = '10'
"""
conn.execute(update_sql)

In [18]:
# Verify upsert
results = conn.execute(f"SELECT * FROM {table_name}").df()
print(results.sort_values("id"))

   id         name create_date                 last_update_time
2   1  Christopher  2020-01-01        2020-01-02 00:00:00+00:00
6  10   Sam-Update  2024-01-01 2025-12-17 17:52:25.858437+00:00
5  11        Jacob  2024-01-01 2025-12-17 17:52:02.547987+00:00
1   2         Will  2020-01-01        2020-01-01 00:00:00+00:00
3   3     Emmeline  2020-01-01        2020-01-02 00:00:00+00:00
4   4         John  2020-01-01        2020-01-01 00:00:00+00:00
0   6         Adam  2020-01-01        2020-01-01 00:00:00+00:00


## 8. Delete records in Iceberg Table

In [19]:
delete_sql = f"DELETE FROM {table_name} WHERE id IN ('10', '11')"
conn.execute(delete_sql)

In [20]:
# Verify delete
results = conn.execute(f"SELECT * FROM {table_name}").df()
print(results.sort_values("id"))

  id         name create_date          last_update_time
2  1  Christopher  2020-01-01 2020-01-02 00:00:00+00:00
1  2         Will  2020-01-01 2020-01-01 00:00:00+00:00
3  3     Emmeline  2020-01-01 2020-01-02 00:00:00+00:00
4  4         John  2020-01-01 2020-01-01 00:00:00+00:00
0  6         Adam  2020-01-01 2020-01-01 00:00:00+00:00


## 9. Additonal example queries

In [21]:
# Filtering (cast VARCHAR id to INTEGER for comparison)
filtered = conn.execute(f"SELECT * FROM {table_name} WHERE CAST(id AS INTEGER) < 10 LIMIT 3").fetchall()
print(filtered)

[('6', 'Adam', '2020-01-01', datetime.datetime(2020, 1, 1, 0, 0, tzinfo=<StaticTzInfo 'Etc/UTC'>)), ('2', 'Will', '2020-01-01', datetime.datetime(2020, 1, 1, 0, 0, tzinfo=<StaticTzInfo 'Etc/UTC'>)), ('1', 'Christopher', '2020-01-01', datetime.datetime(2020, 1, 2, 0, 0, tzinfo=<StaticTzInfo 'Etc/UTC'>))]


In [22]:
# Aggregation on VARCHAR fields
stats = conn.execute(f"SELECT MIN(id), MAX(id), COUNT(*) FROM {table_name}").fetchall()
print(stats)

[('1', '6', 5)]


In [23]:
# String operations
name_stats = conn.execute(f"SELECT COUNT(DISTINCT name), MIN(LENGTH(name)), MAX(LENGTH(name)) FROM {table_name}").fetchall()
print(name_stats)

[(5, 4, 11)]


## 10. Clost DuckDB Connection

In [24]:
conn.close()